In [106]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.integrate import quad
from scipy.optimize import minimize
from datetime import datetime as dt

from eod import EodHistoricalData
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols

import yfinance as yf
import QuantLib as ql
import streamlit as st
import plotly.graph_objs as go
from datetime import datetime,timedelta

In [107]:
def risk_free_curve():
    yield_maturities = np.array([1/12, 2/12, 3/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
    yeilds = np.array([4.70,4.69,4.63,4.42,4.32,4.26,4.18,4.20,4.25,4.30,4.58,4.47]).astype(float)/100
    #NSS model calibrate
    curve_fit, status = calibrate_nss_ols(yield_maturities,yeilds)

    return curve_fit

In [108]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r, option_type):

    # constants
    a = kappa*theta
    b = kappa+lambd

    # common terms w.r.t phi
    rspi = rho*sigma*phi*1j

    # define d parameter given phi and b
    d = np.sqrt( (rho*sigma*phi*1j - b)**2 + (phi*1j+phi**2)*sigma**2 )

    # define g parameter given phi, b and d
    g = (b-rspi+d)/(b-rspi-d)

    # calculate characteristic function by components
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)

    return exp1*term2*exp2

In [109]:
def integrand(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r, K, option_type):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r, option_type)
    if option_type == 'call':
        numerator = np.exp(r * tau) * heston_charfunc(phi - 1j, *args) - K * heston_charfunc(phi, *args)
    elif option_type == 'put':
        numerator = K * heston_charfunc(phi, *args) - np.exp(r * tau) * heston_charfunc(phi - 1j, *args)
    
    denominator = 1j * phi * K ** (1j * phi)
    return numerator / denominator

In [110]:
def heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r, option_type):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r, option_type)

    P, umax, N = 0, 100, 10000
    dphi = umax / N  # Width for integration

    for i in range(1, N):
        # Rectangular integration with midpoint rule
        phi = dphi * (2 * i + 1) / 2  # Midpoint
        numerator = integrand(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r, K, option_type)
        P += dphi * numerator

    if option_type == 'call':
        return np.real((S0 - K * np.exp(-r * tau)) / 2 + P / np.pi)
    elif option_type == 'put':
        return np.real((K * np.exp(-r * tau) - S0) / 2 + P / np.pi)

In [111]:
import yfinance as yf
import numpy as np

def get_stock_price_and_volatility(ticker, period='1y'):
    # Retrieve stock data from Yahoo Finance
    stock = yf.Ticker(ticker)
    
    # Get recent historical data
    data = stock.history(period=period)
    
    # Extract the latest closing price
    latest_price = data['Close'].iloc[-1]
    
    # Compute daily log returns
    data['Log Return'] = np.log(data['Close'] / data['Close'].shift(1))
    
    # Drop NaN values that result from the shift operation
    data = data.dropna()
    
    # Calculate the standard deviation of log returns
    daily_volatility = data['Log Return'].std()
    
    # Annualize the volatility (assuming 252 trading days in a year)
    annual_volatility = daily_volatility * np.sqrt(252)
    
    # Rolling volatility with a 252-day window, annualized
    vol_table = data['Log Return'].rolling(window=252).std() * np.sqrt(252)

    vol_table = vol_table.dropna()
    
    # Check for any remaining NaN values
    if vol_table.isna().any():
        print("There are still NaN values in vol_table")
    else:
        print("No NaN values in vol_table")
    
    # Calculate the volatility of the volatility
    # Add a small epsilon to avoid log(0) issues
    epsilon = 1e-8
    log_returns_vol = np.log(vol_table + epsilon) - np.log(vol_table.shift(1) + epsilon)
    sigma = log_returns_vol.std() * np.sqrt(252)  # Annualized volatility of volatility
    
    return latest_price, annual_volatility, sigma

# Run the function and print the result
latest_price, annual_vol, sigma = get_stock_price_and_volatility('NVDA', period='1y')
print("Volatility of Volatility (Sigma):", sigma)



No NaN values in vol_table
Volatility of Volatility (Sigma): nan


In [112]:
def get_all_options(ticker, option_params):
    stock = yf.Ticker(ticker)
    option_type = option_params[0]  # 'call' ou 'put'
    K = option_params[1]  # Strike price
    r = option_params[2]  # Taux sans risque
    t = option_params[3]  # Temps jusqu'à l'échéance (en années)
    expirations = stock.options
    all_options_data = []

    for expiration_date in expirations:
        options_chain = stock.option_chain(expiration_date)
        
        if option_type == 'call':
            options = options_chain.calls
        elif option_type == 'put':
            options = options_chain.puts
        else:
            return "Invalid option type. Choose 'call' or 'put'."
        options['expiration'] = expiration_date
        all_options_data.append(options)

    options_df = pd.concat(all_options_data, ignore_index=True)
    
    lower_limit = K * 0.85  # 85% de K
    upper_limit = K * 1.15  # 115% de K

    options_df = options_df[(options_df['strike'] >= lower_limit) & (options_df['strike'] <= upper_limit)]

    unique_strikes = options_df['strike'].unique()
    unique_times = options_df['expiration'].unique()

    days = int(t * 365)
    today = datetime.now()
    expiration_date = today + timedelta(days=days)

    if K not in unique_strikes:
        new_row_K = options_df.iloc[0].copy()
        new_row_K['strike'] = K
        new_row_K['expiration'] = expiration_date.strftime('%Y-%m-%d')
        new_row_K['impliedVolatility'] = np.NaN 
        #options_df = options_df.append(new_row_K, ignore_index=True)
        options_df = pd.concat([options_df, pd.DataFrame([new_row_K])], ignore_index=True)

    if t not in unique_times:
        new_row_t = options_df.iloc[0].copy()
        new_row_t['strike'] = K
        new_row_t['expiration'] = expiration_date.strftime('%Y-%m-%d')
        new_row_t['impliedVolatility'] = 0
        #options_df = options_df.append(new_row_t, ignore_index=True)
        options_df = pd.concat([options_df, pd.DataFrame([new_row_t])], ignore_index=True)

    options_df = options_df.sort_values(by=['strike', 'expiration'], ascending=[True, True]).reset_index(drop=True)

    return options_df[['strike', 'expiration', 'lastPrice', 'bid', 'ask', 'impliedVolatility']]

options_data = get_all_options('NVDA', ['put', 150, 0.05, 1])
print(options_data)

#options_data.to_csv("C:\\Users\\tomto\\OneDrive\\Documents\\Excel\\output_option_data.csv")

     strike  expiration  lastPrice    bid    ask  impliedVolatility
0     128.0  2024-11-15       0.12   0.11   0.12           0.585942
1     128.0  2024-11-22       0.94   0.92   0.96           0.634769
2     128.0  2024-11-29       1.26   1.24   1.28           0.559575
3     128.0  2024-12-06       1.66   1.59   1.67           0.524419
4     128.0  2024-12-13       2.05   2.00   2.07           0.505864
..      ...         ...        ...    ...    ...                ...
580   172.0  2025-06-20      80.91    NaN    NaN           0.000000
581   172.0  2025-12-19      41.50  41.15  41.55           0.416388
582   172.0  2026-01-16      48.80  41.80  42.20           0.412939
583   172.0  2026-06-18      68.69  52.50  67.50           0.594578
584   172.0  2026-12-18      64.15  58.00  68.00           0.554860

[585 rows x 6 columns]


In [113]:
import numpy as np
import pandas as pd
from scipy.interpolate import griddata

def create_volatility_table(options_df):
    # Conversion de la colonne 'expiration' en type datetime
    options_df['expiration'] = pd.to_datetime(options_df['expiration'])
    
    # Création d'un tableau de volatilité avec strikes en index et expirations en colonnes
    volatility_table = options_df.pivot_table(
        index='strike',
        columns='expiration',
        values='impliedVolatility'
    )
    
    # Remplacer les zéros par NaN pour ne pas interférer avec l'interpolation
    volatility_table.replace(0, np.nan, inplace=True)

    # Extraire les indices (strikes et expirations) et les valeurs
    strikes = volatility_table.index.values
    expirations = volatility_table.columns.values

    # Préparer les points connus (non-NaN) pour l'interpolation
    points = []
    values = []

    for i, strike in enumerate(strikes):
        for j, expiration in enumerate(expirations):
            if not np.isnan(volatility_table.iat[i, j]):  # Ignorer les NaN
                points.append((strike, expiration))
                values.append(volatility_table.iat[i, j])

    # Vérifier si suffisamment de points sont disponibles pour l'interpolation
    if len(points) < 3:
        raise ValueError("Pas assez de points non-NaN pour l'interpolation.")
    
    # Créer une grille complète pour tous les points (strikes, expirations)
    grid_x, grid_y = np.meshgrid(strikes, expirations, indexing='ij')

    # Appliquer l'interpolation bilinéaire (avec méthode 'nearest' ou 'cubic' pour tester)
    try:
        interpolated_values = griddata(points, values, (grid_x, grid_y), method='linear')
    except Exception as e:
        #print(f"Erreur avec interpolation bilinéaire: {e}")
        #print("Essai avec méthode 'nearest'.")
        interpolated_values = griddata(points, values, (grid_x, grid_y), method='nearest')

    # Créer un DataFrame avec les valeurs interpolées
    interpolated_table = pd.DataFrame(interpolated_values, index=strikes, columns=expirations)

    return interpolated_table

# Utilisation de la fonction avec vos données
try:
    volatility_table = create_volatility_table(options_data)
    #print(volatility_table)
except Exception as e:
    print(f"Erreur : {e}")





In [114]:
def create_price_table(options_df):
    # Conversion de la colonne 'expiration' en type datetime
    options_df['expiration'] = pd.to_datetime(options_df['expiration'])
    
    # Création d'un tableau de volatilité avec strikes en index et expirations en colonnes
    price_table = options_df.pivot_table(
        index='strike',
        columns='expiration',
        values='lastPrice'
    )
    
    # Remplacer les zéros par NaN pour ne pas interférer avec l'interpolation
    price_table.replace(0, np.nan, inplace=True)

    # Extraire les indices (strikes et expirations) et les valeurs
    strikes = price_table.index.values
    expirations = price_table.columns.values

    # Préparer les points connus (non-NaN) pour l'interpolation
    points = []
    values = []

    for i, strike in enumerate(strikes):
        for j, expiration in enumerate(expirations):
            if not np.isnan(price_table.iat[i, j]):  # Ignorer les NaN
                points.append((strike, expiration))
                values.append(price_table.iat[i, j])

    # Vérifier si suffisamment de points sont disponibles pour l'interpolation
    if len(points) < 3:
        raise ValueError("Pas assez de points non-NaN pour l'interpolation.")
    
    # Créer une grille complète pour tous les points (strikes, expirations)
    grid_x, grid_y = np.meshgrid(strikes, expirations, indexing='ij')

    # Appliquer l'interpolation bilinéaire (avec méthode 'nearest' ou 'cubic' pour tester)
    try:
        interpolated_values = griddata(points, values, (grid_x, grid_y), method='linear')
    except Exception as e:
        #print(f"Erreur avec interpolation bilinéaire: {e}")
        #print("Essai avec méthode 'nearest'.")
        interpolated_values = griddata(points, values, (grid_x, grid_y), method='nearest')

    # Créer un DataFrame avec les valeurs interpolées
    interpolated_table = pd.DataFrame(interpolated_values, index=strikes, columns=expirations)

    return interpolated_table

# Utilisation de la fonction avec vos données
try:
    price_table = create_price_table(options_data)
    #print(price_table)
except Exception as e:
    print(f"Erreur : {e}")

In [115]:
# Convert our vol surface to dataframe for each option price with parameters
def table_vol_to_dataframe(volatility_table):
    volSurfaceLong = volatility_table.melt(ignore_index=False).reset_index()
    volSurfaceLong.columns = ['strike','maturity', 'price']
    today = datetime.today()
    volSurfaceLong['years_to_maturity'] = volSurfaceLong['maturity'].apply(lambda x: (x - today).days / 365)

    # Calculate the risk free rate for each maturity using the fitted yield curve
    volSurfaceLong['rate'] = volSurfaceLong['years_to_maturity'].apply(risk_free_curve())

    return volSurfaceLong

volSurfaceLong = table_vol_to_dataframe(price_table)
print(volSurfaceLong)


      strike   maturity  price  years_to_maturity      rate
0      128.0 2024-11-15   0.12           0.010959  0.047160
1      129.0 2024-11-15   0.12           0.010959  0.047160
2      130.0 2024-11-15   0.12           0.010959  0.047160
3      131.0 2024-11-15   0.15           0.010959  0.047160
4      132.0 2024-11-15   0.15           0.010959  0.047160
...      ...        ...    ...                ...       ...
1073   168.0 2027-01-15  48.50           2.178082  0.041952
1074   169.0 2027-01-15  48.50           2.178082  0.041952
1075   170.0 2027-01-15  48.50           2.178082  0.041952
1076   171.0 2027-01-15  48.50           2.178082  0.041952
1077   172.0 2027-01-15  48.50           2.178082  0.041952

[1078 rows x 5 columns]


In [116]:
def plot_volatility_surface(volatility_table, strike, expiration_date):
    volatility_table.index = pd.to_numeric(volatility_table.index)
    volatility_table.columns = pd.to_datetime(volatility_table.columns)
    today = datetime.now()
    days_to_expiration = [(exp - today).days for exp in volatility_table.columns]

    days = int(expiration_date * 365)
    today = datetime.now()
    expiration_date = today + timedelta(days=days)
    expiration_date = pd.to_datetime(expiration_date).strftime('%Y-%m-%d')

    # Retrieve the volatility value at the user-specified point if it exists in the table
    try:
        implied_vol = volatility_table.loc[strike, expiration_date]
        user_point = True
    except KeyError:
        implied_vol = None
        user_point = False

    # Create the surface plot
    surface = go.Surface(
        x=volatility_table.index,
        y=days_to_expiration,
        z=volatility_table.values.T,
        colorscale='Viridis',
        colorbar=dict(title='Implied Volatility')
    )

    # Create a trace for the user-specified point in red, if it's within the surface data
    if user_point:
        #st.success("We found the point")
        point_trace = go.Scatter3d(
            x=[strike],
            y=[days],
            z=[implied_vol],
            mode='markers',
            marker=dict(size=10, color='red'),
            name='User Point'
        )
        data = [surface, point_trace]
    else:
        st.warning("The specified strike or expiration date is not within the range of the volatility table.")
        data = [surface]

    # Create the layout
    layout = go.Layout(
        title='Implied Volatility Surface',
        scene=dict(
            xaxis_title='Strike Price',
            yaxis_title='Days to Expiration',
            zaxis_title='Implied Volatility',
            xaxis=dict(gridcolor='rgb(255, 255, 255)', zerolinecolor='rgb(255, 255, 255)', showbackground=True),
            yaxis=dict(gridcolor='rgb(255, 255, 255)', zerolinecolor='rgb(255, 255, 255)', showbackground=True),
            zaxis=dict(gridcolor='rgb(255, 255, 255)', zerolinecolor='rgb(255, 255, 255)', showbackground=True)
        ),
        width=900,
        height=700,
        margin=dict(r=10, l=10, b=10, t=40)
    )

    # Create the figure
    fig = go.Figure(data= data, layout=layout)

    return fig

plot_volatility_surface(volatility_table,150,1)
#plot_volatility_surface(price_table, 150,1)

In [117]:
# This is the calibration function
# heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)
# Parameters are v0, kappa, theta, sigma, rho, lambd


# Define variables to be used in optimization
def calibration_params(volSurfaceLong, latest_price, vol_histo, sigma):
    S0 = latest_price
    r = volSurfaceLong['rate'].to_numpy('float')
    K = volSurfaceLong['strike'].to_numpy('float')
    tau = volSurfaceLong['years_to_maturity'].to_numpy('float')
    P = volSurfaceLong['price'].to_numpy('float')

    
    params = {"v0": {"x0": np.sqrt(vol_histo), "lbub": [1e-3,1]},
          "kappa": {"x0": 3, "lbub": [1e-3,5]},
          "theta": {"x0": 0.05, "lbub": [1e-3,0.1]},
          "sigma": {"x0": sigma, "lbub": [1e-2,1]},
          "rho": {"x0": -0.8, "lbub": [-1,0]},
          "lambd": {"x0": 0.03, "lbub": [-1,1]},
          }

    x0 = [param["x0"] for key, param in params.items()]
    bnds = [param["lbub"] for key, param in params.items()]

    return S0, r, K, tau, P, params, x0, bnds

def SqErr(x, S0, P, K, tau, r, x0, c_p):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]

    # Decided to use rectangular integration function in the end
    err = np.sum( (P-heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r, c_p))**2 /len(P) )

    # Zero penalty term - no good guesses for parameters
    pen = np.sum( [(x_i-x0_i)**2 for x_i, x0_i in zip(x, x0)] )

    return err + pen

In [118]:
S0, r, K, tau, P, params, x0, bnds = calibration_params(volSurfaceLong, latest_price, 0.7, 0.5)

def f_minimize(S0, P, K, tau, r, x0, tol, bnds, c_p):
    result = minimize(SqErr, x0, args=(S0, P, K, tau, r, x0, c_p), tol=1, method='SLSQP', options={'maxiter': 1000}, bounds=bnds)
    v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
    return v0, kappa, theta, sigma, rho, lambd

In [119]:
v0, kappa, theta, sigma, rho, lambd = f_minimize(S0, P, K, tau, r, x0, 1, bnds, 'put')
v0, kappa, theta, sigma, rho, lambd

(0.0010132738256130496,
 4.999974464229988,
 0.0010000000000000005,
 0.8895210835712601,
 -0.9999974690502902,
 0.999987985943902)

In [120]:
def add_result(volSurfaceLong, S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    heston_prices = heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r,'put')
    volSurfaceLong['heston_price'] = heston_prices
    return volSurfaceLong

In [121]:
volSurfaceLong2 = add_result(volSurfaceLong, S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)
print(volSurfaceLong2)
#volSurfaceLong.to_csv("C:\\Users\\tomto\\OneDrive\\Documents\\Excel\\output.csv")

      strike   maturity  price  years_to_maturity      rate  heston_price
0      128.0 2024-11-15   0.12           0.010959  0.047160    -19.786824
1      129.0 2024-11-15   0.12           0.010959  0.047160    -18.479359
2      130.0 2024-11-15   0.12           0.010959  0.047160    -17.317248
3      131.0 2024-11-15   0.15           0.010959  0.047160    -16.391590
4      132.0 2024-11-15   0.15           0.010959  0.047160    -15.658154
...      ...        ...    ...                ...       ...           ...
1073   168.0 2027-01-15  48.50           2.178082  0.041952      8.152237
1074   169.0 2027-01-15  48.50           2.178082  0.041952      8.659009
1075   170.0 2027-01-15  48.50           2.178082  0.041952      8.738708
1076   171.0 2027-01-15  48.50           2.178082  0.041952      8.575221
1077   172.0 2027-01-15  48.50           2.178082  0.041952      8.332905

[1078 rows x 6 columns]


In [122]:
import plotly.graph_objects as go
from plotly.graph_objs import Surface

In [126]:
def plot_mesh(volSurfaceLong):
    fig = go.Figure(data=[go.Mesh3d(x=volSurfaceLong.maturity, y=volSurfaceLong.strike, z=volSurfaceLong.price, color='mediumblue', opacity=0.8)])

    fig.add_scatter3d(x=volSurfaceLong.maturity, y=volSurfaceLong.strike, z=volSurfaceLong.heston_price, mode='markers', marker=dict(size=5, color='red'))

    fig.update_layout(
        title_text='Market Prices (Mesh) vs Calibrated Heston Prices (Markers)',
        scene = dict(xaxis_title='TIME (Years)',
                    yaxis_title='STRIKES (Pts)',
                    zaxis_title='INDEX OPTION PRICE (Pts)'),
        height=800,
        width=800,
    )

    return fig

print(plot_mesh(volSurfaceLong2).show())


None
